In [62]:
import pandas as pd
import numpy as np
import os
import requests
import re

In [53]:
import warnings
warnings.filterwarnings('ignore')

# Concatenate yearly song releases

In [4]:
year = [i for i in range(2012, 2022)]

sr = pd.read_csv('song_release/%s.csv' %year[0], index_col=False, names=['title', 'artist'])
sr['year'] = [year[0]] * len(sr)

for i in year[1:]:
    sr_i = pd.read_csv('song_release/%s.csv' %i, index_col=False, names=['title', 'artist'])
    sr_i['year'] = [i] * len(sr_i)
    sr = pd.concat([sr, sr_i], ignore_index=True)

# strip whitespaces
sr['title'] = sr['title'].str.strip()
sr['artist'] = sr['artist'].str.strip()
sr

,title,artist,year
0,(I Called Her) Tennessee,Tim Dugger,2012
1,"10,000 Reasons (Bless the Lord)",Matt Redman,2012
2,100 Proof,Kellie Pickler,2012
3,101,Alicia Keys,2012
4,110%,Jessie Ware,2012
...,...,...,...
20032,You're To Blame,Mammoth WVH,2021
20033,"Young, Black And Beautiful",Chris Pierce,2021
20034,Younger Me,Brothers Osborne,2021
20035,Your Power,Billie Eilish,2021


In [22]:
sr['track_id'] = [np.nan] * len(sr)
sr['release_date'] = [np.nan] * len(sr)
sr['spotify.name'] = [np.nan] * len(sr)
sr['spotify.artists'] = [np.nan] * len(sr)

## Get Spotify track ID

In [44]:
def getArtistNameList(result_artists):
    artists = []
    for a in result_artists:
        artists.append(a['name'])
    return artists

In [92]:
CLIENT_ID = '6ced8cf12e0c470f9d9dd25a75d4ec26'
CLIENT_SECRET = '4ce366cebd7c42f2ac59bf96ca9f32b9'

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

### Search directly by title and artist

In [36]:
try:
    start_idx = sr.loc[~sr['track_id'].isnull(),:].index.values[-1] + 1
except:
    start_idx = 0
    
for i in range(start_idx, len(sr)):
    title = sr['title'][i]
    artist = sr['artist'][i]
    
    query = 'track:"%s" AND artist:%s AND year:%s&type=track&limit=1'%(title,
                                                                        artist,
                                                                        sr['year'][i])

    r = requests.get(BASE_URL + 'search?q=' + query, headers=headers).json()

    try:
        results = r['tracks']['items'][0]

        artists_i = getArtistNameList(results['artists'])

        for a in artists_i:
            # deal with middle name -> match only first and last name
            if (a.lower() in artist.lower()) or (artist.lower() in a.lower()) or\
            ((artist.split()[0].lower() in a.split()[0].lower()) and (artist.split()[-1].lower() in a.split()[-1].lower())):
                print(i, results['id'])
                sr['track_id'][i] = results['id']
                sr['release_date'][i] = results['album']['release_date']
                sr['spotify.name'][i] = results['name']
                sr['spotify.artists'][i] = artists_i
                break
    except:
        pass

2849 1aMvViuAn7i8gRapm8C0p8
2851 3cEwW0OE6GhpufUNdOFKna
2852 7hiBhOh4s8G0xmGYBAjyuw
2853 3iKmdCDZK4XmqxIA9fVVGs
2854 1KA41c2rMefD2Co4YEU6CN
2855 2ICDzsIPWJZk0YR0tPFDlQ
2856 5Ck3e6z1bcSCes5gdo0Ddt
2857 6yNT4GWkOud1ea7V2UxW2P
2858 1eFiehZXXGbxfkZx9mraUj
2859 5enJVxmVHb0ncSqG3zLIcT
2860 4gBoCxm0M1bueAO3XCojP4
2861 3MukYE0LyQELSCjcZIlcTs
2862 01ZtY0A5Z7jMnnZUt0NHGo
2863 2zA4Zy2oonhVXzoRglnJFz
2865 6CEcpl4x9xv0KPe6o2xiDA
2867 3D7izPzeEcz7lBhcCZJccl
2868 5qrkLSD1Lf1K9ZwdrMTpzN
2869 6Xk7PnitV9jCRorWt2LiVZ
2871 2bN0VoXCagktb6yAi9942I
2872 74k27gGRRskIdlZ4aiN2cS
2873 5T7ZFtCcOgkpjxcuaeZbw0
2874 4KGfgBUBuHoYIfh02RUX5p
2875 5Pnny78GESkBSLnxFmhRYZ
2877 1wDz2KEtxAbyKj4AoA02ms
2878 4qG3Um3jXlQnaQV1yVo3it
2880 6bxUnsSGZCmoHHU5auwtps
2881 0k1xMUwn9sb7bZiqdT9ygx
2882 4oFEus9krS5v57wAcggcEz
2884 51pmxyLDp9VS592lRJ5Ru4
2885 4RcriZ7GVTxqzAWaCLP4nT
2886 2ZTECZ6n3jCqBWxZcQzuvb
2888 2xLOMHjkOK8nzxJ4r6yOKR
2889 10J2smxgutWDGTeFy4kYp8
2890 1VZ4S2A6eUdp1d3WxH4xR8
2891 3lQIYJk2VrpvzZFOPR3ZPD
2892 4mfMwQoMdOeW3u2

3215 2yKczCmnpWYuIJuel6M0eV
3218 79zhandwZPrlTWemEpIgxy
3219 01zPnSlKCw9WLbY0kvNep0
3221 1XTp9AoHDoCQMmMWH0ch8M
3222 7mDFMh8pghY0MRToYb01eI
3223 6twW4ma6w0mOeejejPK0nY
3224 10f0hrbCBiQk9STrFG3EU7
3225 2CINrkPbwEWt2bWSsh7wrY
3228 7vQHgaUJCODPVaAhJIJyjz
3229 1eQc5bxMTKOmfrotjbygkz
3230 0BQV07dSwLltMlLDo47ckp
3231 29Zj1r8cek15xe08vNUOLy
3232 44CrEafj3AU5cquvEi1tBg
3233 7zAyi8yH16FZXbd6L8A0KC
3234 4bPN0e4MXUBymOsLrqJsvU
3235 5NTCXRjhs6qyn9EJmpEITh
3236 136SnNOBqkCzra05ZW9U1n
3237 5JjNoc9KabK5WASRCzVWAw
3238 1CcSjozRjEayDCUFlvhUrj
3239 3P5pqmsfSJn4hE7i3Zgcvw
3240 2dM0bOcE7IyFzXnnNx262Y
3241 6ZUITkPPMScmzjW4jFkBn6
3242 5FVd6KXrgO9B3JPmC8OPst
3243 4Ma6fKktonx1Tu7Lzljcz5
3244 6n2GPvMJtFZ4ckoc9EWHMx
3246 5XKXMWPACPq51OiqzxenZo
3247 5VJRSXqHca3yAsCEISymlc
3249 5SP3MQuYB2tqhhFx0Ukk3s
3250 36c4JohayB9qd64eidQMBi
3251 0BFNvBZf8ETEzQ4pxVZhpz
3252 1MDogqDrStFSO9cdUTUFem
3253 1ofcBtqGQIc23Nk440dNqd
3254 4OyfdJiHM6sbfkFmRLrygg
3262 1vjx9wZEyhhM1VRmIoDiRk
3273 22pWSUvRllTCBOCB4EZ5n2
3274 7t3yEWJXS4ZZE64

3579 5ylBnaUYbWixo1ptaaWP2P
3580 2A2TYnOKMiulM9DMFxZ2xv
3581 52m3svaHBHJIySJGaUL4IB
3582 3qGl6bfdUNjgDG3fvHZVAu
3583 15NGPktDl1wh3kY4rMS9TH
3584 6qfIwUb7F1cdZUq7ECiEPx
3585 42hwuSY031D5i1EaUS5MU4
3586 1SniLCGHvhHJAiGe3hLboq
3587 0N8hlnrATQx3yRxC7vrx8B
3588 0M8MIyYKkq2ETYuM57LzI0
3590 7vCZG7hYtt4JOUOYmKVg1S
3591 5nPOM0nFRAJ3XI8xCxNAzO
3592 14i7RZBBM2aTLHSnAX31yO
3593 4cTVAIFH1zGNQFFRFMCzsd
3594 4fuFTnTJpL78ZvVDFKdmoV
3595 4iQLJCvthLGQDXFYkfAXbG
3596 2BnfMbmWbfqi7ac5y7GHXn
3597 73Q1GbrFMAQw8UEYinBwIV
3599 5anCkDvJ17aznvK5TED5uo
3601 3Mg3jyQgJRBxMMvwSFbC1z
3602 3Z2I6PBNAKLyRiJyF1H4N5
3603 0LEycyIxbAwwB9U0GgMx39
3604 00CqEmnPLFKDhAb3cuu6Cs
3605 1v2Ywn0extT3YQDqnh8RzS
3607 6AY1M1akbsVaQN3ATVyzH7
3608 1Lv35CbXlVhZdoK7O9KjGI
3609 5Lf9rCqReE9Ri4tZvLRLuy
3611 5ZikX4TJeBDVaReZ5vGzl2
3613 3U3BXlwNTkCx7lW9m1H9RY
3614 6OlM7E39dtzyWcV1SPbmFx
3615 3OFD6HAlPiHqy2XoPNaVMR
3616 4PP1na4boWVqVWZBwZF1vF
3617 3Jx94ykL3OlINrwaheAT2c
3618 3TCNa0C0VqGpKlI2XjR2RB
3619 3AX4HIRJY6GT6tgMkO5987
3620 0WpNKnxmUreDeAe

3974 3F821OqRN1oGQg8FKC0S5U
3975 3NxAxBnK66q0ZpIWeH9xlp
3976 2la10ufvOkSlJ4jCRp2NXl
3977 52gvlDnre9craz9dKGObp8
3978 00gIstLkmQGkJP7sxdKcXC
3980 5WcsKj0O2FG8etJAfnogBa
3981 0hW72l6tZgSY7Od2OUVDIC
3983 77wRn640iIPc3w0FoBRzOn
3984 2VBluJiNj9bKohWX0dY1Lh
3985 40QYehV6XB0p0ZJG1nLLDK
3986 4cJIcaWrNiy7AqownBIbwf
3987 2m32wOGkiV8ltxXvwwVeao
3989 0AIEEspWy0KQ6m3fGWGaib
3990 4hLmZWEHm9VjjwxOaYVJOi
3992 70vyPOk2VdMN53oOsLtPYJ
3993 3Q9YjVy5ql7BDL4OoEAE78
3994 6ekvzkGg31TOQhHIFEd55x
3995 6lwwiO35HcxcSfOQNgMITz
3996 2wdRWIsv9QnDFO9trsniXw
3997 2Hb4cQNAvSgoaWx25QlDOw
3998 2HKYzss2hIpWiObSDqlXIa
3999 6RjZ1VwsW4g93aX3dyu01M
4000 03YqVBjqIVJbxYcW0c5S8E
4001 121oeMXoYl1zJjo0PGeALU
4003 1mavQ4WCzXSeL2Dm5DS4GQ
4004 7IEU0ixilAPy6lJFtmm3Lq
4005 5Uc37Y6FijZcM538PqqkWd
4006 5CFNWVYldVhbAq4Y2fxBNV
4007 0QluZ9YIqioL7yscY7LdV6
4008 0qcr5FMsEO85NAQjrlDRKo
4009 69BdRU3cyxzGCEpugK3Ghh
4011 2o0V0AATmWmCmMOAFWDuEV
4012 6BxljNYeBfXNcWQcdt1KVu
4013 4L1MrFFkGrs5Mxkj7Xamti
4014 6WouUO5WEg0ncfGn6dkB4I
4015 1nmRK4GskYDqBum

4323 2HWQgbw7JW7Mvzu99GoiXB
4324 1PglrPaaUHsfp2Cka5CLdK
4325 7aK5rlTVxSgcqShPMI1TTH
4326 4xcxPcWkdoAw6g5WfXy9Gy
4327 3nB82yGjtbQFSU0JLAwLRH
4328 5Axwf04zxc1LTPOwxr75Bx
4330 0JmlzuOyMCaMFMaSutmf3Q
4331 2jfk3m8nq5IvB35s2xIs1O
4332 12EM4ex0heaKMgeClWO3Ec
4333 72bWFl3uRvoAS2l4c8Z6su
4334 3m16spqwp6OJxqN4IXcsU7
4335 0TQ8ZHFhrbH9I0WRjIzOCQ
4336 5V6qm8pM4nJJ3gm9ajxN0c
4337 04bqNCd6eN1qjldAG483qA
4338 0qbllmDiMUIhQS64qUkTjf
4340 30Co9eN7JHPf1i2wEyVSMJ
4341 1dMOkSlIgQEgIKFTz4LqL6
4342 0LBz07gAdgUukYvpFyW87y
4343 0D92jKDWCYyJcADZjHT2ux
4345 4tqxfer3xKyzl4sRFxRDNv
4346 3ptAHA8QSbBeBx01t8f0ca
4347 1kO615U99ZkJP2RSK9zR8x
4348 1q5KvzDyGurl95wrFj9Mxy
4349 1yzTwSlAy67xC2vptWBBag
4351 4RZBJgDzmIDA3pzGpg5icG
4352 64gyqTtDDel7B2NoEkOa0A
4353 7vDpua1zIjG5UVmBzmxnUP
4354 79npfYK6N5RnBc1jiaoHCr
4356 3ri4y4r1BFc9YiPNgFXeGW
4357 3VeW0pO2pae144rAX2oXPP
4358 5Mw9bXG1dLNhbjofkVS2oR
4359 5uAKVmWGpcdZyOB0DUlQCM
4360 0pf50dEOfMpgS1DS79b5a9
4361 7BNDyzwDboNRR2wmd7GSew
4363 4XYyMYi9V9CmzD9ZGZVA0D
4364 2VDkcDVx9P9DY4w

...

14803 6n3HGiq4v35D6eFOSwqYuo
14804 21RCuMVSlblTmyYm2DnPCR
14805 5jS1YDipyHQzxYRcfizhPS
14806 3Ho6BPy19x7QcFYW9e877B
14807 1G1r2ectZOGeOkZmKtAP3m
14809 4nrg60oftaZJqgKvGd9eaB
14812 4sUD40pfamVQwelkx6bNK0
14814 0k6C6Am1iAZY6BfW104DAZ
14815 6uxid9G2Ud7jzbsLOElM39
14816 28MPfAfqhRiBJLzNCHGHM3
14817 2CiuGhZPtEjBAK1aS1oYDJ
14818 6mBlft6IhCJwPNKhnbb0wK
14819 1vwEgKpkdY63nX0jrrYj9X
14820 5pg8E1J3N5yZw2h6cYl9l7
14821 6EgtWbbEwTMQruuK0W9CYm
14822 3zDnrvd9Uw6iHVqOgauuw6
14823 0LPiTwaf28kZyrDTQPjuV3
14824 0aFXE1GU6jYnz8MkKyArZw
14825 34oB5r0lcN3fYWCs2uA1k5
14826 2ZQiDvZgYkUYsyMENSL5xr
14827 3tdCUI10fh1LrwWKXn7Db3
14828 11oSlSG6IsITAfSVRp7uUc
14829 1qFky8IUQfzPFhkAnmGGEa
14832 5xoNx4guLOeGsuFxuqTord
14833 56OhItQ6ACZrsi8FhR7nHd
14834 3BEJxxZPPJKCTyNjiaxmny
14835 0M9dW4KlvggpF2PtCF84in
14836 3HWlJu9sa7viLF72eMnrlP
14837 66bVLKIOwOHbHcnO7nLKpL
14838 1IV4KjKmVgk3tn1Ffnbtuh
14839 5hbttdGGhvN9PUZDwgCk67
14840 5N6vFQqJJkee6jTm7wct5C
14841 5yDJpu0xh0d1w13gXaE3lS
14843 1ZUsd5fW7uuXnrbgVIWYb9
14844 7Ekqomzu

15133 5ZQYl1nIY1s1FvfKNkPeJg
15134 4cqpXFj1ZeuInVprRQ0FDx
15139 2dhJM2skHC9D41gHj34Kgw
15140 1KeZgPUr54C8iz3FjqzVoz
15141 3C3lycSM0ZeVzLtUsruBao
15142 1q1czFTQvKFTlNGXO4qLUj
15143 7xild9zoXvcMrXa11nmOKK
15144 5Z3GHaZ6ec9bsiI5BenrbY
15145 2fGxYYMQbVqgIdaqfz2OwA
15147 6HyZGZqZEly1UYW9d6CFxV
15148 4c2W3VKsOFoIg2SFaO6DY5
15149 4D4xJ4LyDXBWIf9thFSMAJ
15150 5RmOHOEZxoMryrW3WiE9Jg
15151 2hl6q70unbviGo3g1R7uFx
15152 7pse475uICmWRY5hEkvPvI
15153 3BDxv4ORzOpXAsa4lWVl4i
15154 0x64VU0L9lsdVU12AHp01L
15155 0AluA5RNsa4Cx6XRhf2hWZ
15156 1PhwjWzUJRNcKBhgVhxG8l
15157 0CZ8lquoTX2Dkg7Ak2inwA
15158 2lVhH2L2b8PkagS3wDZS7D
15160 05mAIVLkIWc2d1UBYZBCp8
15161 2BgEsaKNfHUdlh97KmvFyo
15162 0BmJxvH6PDUxZmwB6dIZag
15163 6irE4Fbjbry3czv7mTxVSB
15164 1tNJrcVe6gwLEiZCtprs1u
15165 3e7Y6sfFlIdBMJhX7wpqVO
15166 3Ua3boF2tSlVSsW2mQ0GCG
15167 5vXX00R8hzUTNJxxEye2lF
15168 6JyEh4kl9DLwmSAoNDRn5b
15169 5KO53tfiVN6daF4w6twc63
15170 11kDth1aKUEUMq9r1pqyds
15173 6tarvNiKnEjYMj1VZhlDqR
15174 6qVwzCWDC6Kiatl7v2it7D
15175 0nox76Vx

15448 4fhKaRCxbWXA3BwAr3Zxjx
15449 2u0vDb6asmmWM4LepQ3196
15450 4lQsB3ERTWSNaAN1IkuNRl
15451 0dApI39mMqaXSbSsq4J7kI
15452 0lZ3yLWp2pAWExImZVSGJ3
15453 0kDPQ4qwk2WLfg7BcZi4Tm
15454 7xpfgUe2sxoBx7CFygbTPg
15455 5PqnsLEMlnsAfAKBSfjUB5
15456 0Y7di9ZCmB6Elad5WISaPc
15457 6fIjnWrv46njJHLDAY2JdC
15458 3VHNtJNVPazkHGpeGgpoOd
15459 1wnVYFmT4wXCFjCzp1CrAN
15460 1vHeu2j2MpfMqqvcXKLtic
15461 0awvW9Pcq2skIqVOUdk5wX
15463 2hEYslbdYWFSIA6JboUvnR
15465 1uWZCPH9z8cFkxCtHUQi6b
15466 22mQXNE0nCuWq4yOwcadIn
15468 4haJoBPbPj9QFKNpp9e8kx
15469 1DNRyoXyqj4b3sbtPjfGx3
15470 7nJZ9LplJ3ZAyhQyJCJk0K
15471 5jmUHxvoxfA66wp4bN8rjc
15472 5ZHPzr7sZJLmBIyfvAIeLN
15473 1hkm127hnTSa2xjGNw8IAE
15474 1O9DWF3578RHMxZg2nLPeM
15475 1zOVEJ16YD8QjiZSBWdblb
15476 56RfNBJGUgL1ZFCB1KEJrQ
15477 1dOioyTkknRYIjRlTtN8R5
15478 6oIjJi274wP86uqFxQRlKV
15479 11FvComd16zZX61DrlP3o1
15481 67BPY7ijAvZR9xbn5YwF4u
15482 1eQN2B11BU0sIqBGAGnwsH
15494 6G8kHiVZ1jW7vHMPVRNZU0
15496 0BteLcimTI0usYZ2L2zQaZ
15497 1lsBTdE6MGsKeZCD6llNu7
15498 2D9ABkdy

15775 5p7GiBZNL1afJJDUrOA6C8
15776 7apizLyhMRutg3ccXvr2qv
15777 1SR5WPj04EEqL8mWQ6a3bO
15778 7iPlcFvOMOzt6v0QvcAueZ
15779 3UeTr9z3nVsPVg58IKGHcT
15780 46OF5m5zH7adEEdHQrpEC2
15781 1hEiVb61xXfCc9JL4cJRQP
15785 1f5PNhkNgUpvDEeZfcIlO1
15786 14boA065HMHWvqyPPEeN4y
15792 2m6FcfDqnTG5fNKu3O8jkA
15793 5Yfq9uugLFchtFymQQWxgt
15794 1L7HnGxcNdScMVnjc3XRK0
15795 6j6D0RMPy68MP0xJ5ESHbs
15797 2xvVPQVsw51FFFocmAe6Ek
15798 58q2HKrzhC3ozto2nDdN4z
15800 2EznBGrlmx9wBeYgyDojsA
15801 4S8d14HvHb70ImctNgVzQQ
15802 32UJH1F38AMSjZilQyBzOE
15803 6bbCmkbeIFlq3ZWHXz3MyR
15804 0yhxBvedRdGxsPZHJNI4VA
15805 73JliUle2q8xwGxWjOquMj
15806 1cKHdTo9u0ZymJdPGSh6nq
15816 0iCUBrpsGH8xtIBE9Yy0iZ
15817 5r8c96Zn0ZNHRecVKpFf23
15818 2AIWoHr9DF6y4KALCBKWQS
15820 5TKoiKYuXhHG90uESCz4dg
15821 0bhBfW4mSRLILc32S4KugQ
15822 5Ie8L8TDPKCxykYzbNRKA4
15823 51KMpn0p5Vb7XVdV7mWau5
15825 1ue1rcTQ3kw47AQR90OYna
15827 1TuSAhOzTqoFf7BZxBl6nz
15829 7oBbn0yFlspuZrNP8Mi0yX
15830 2IuazAsyvbb3fFZb0Uvsmk
15831 2CEjWJ4lPiodKXXS2bhVyP
15832 2QZ7WLBE

16100 2xECSFHPgCyElhTVRaHVYP
16101 55yu6z1MoEyGXJhSBbyDIi
16102 2t2cA7KQ3v8qSjnKmqSSZB
16103 1gyFrirLHkX1u6v2DQ9I3b
16104 04MLEeAMuV9IlHEsD8vF6A
16105 1Fr7OlVZrc3mDvru8j2cFi
16106 2qT1uLXPVPzGgFOx4jtEuo
16107 6reMZLKGViMsTo4GKSieII
16108 2P5yIMu2DNeMXTyOANKS6k
16109 2UNSjAMTbEYRVgj0VscPEA
16110 1njVROLMK2Hgj8WJyWwAV1
16111 0TlLq3lA83rQOYtrqBqSct
16112 0evt4UZbdhnHtcAnxkm6A1
16114 4XEQgLdAE4KRIsSNP9dRWX
16115 6TWoOOvqyMHBUuHpNS5Rre
16116 28FGV3ORH14MYORd7s5dlU
16117 2sF0uYdAwFsT9Iq0h2DDU9
16118 7sy21h4cRfMynPB1xP5OMr
16119 1BGIZCaDu29MK8DsykL7Hb
16120 27rdGxbavYJeBphck5MZAF
16122 15T8BgkXVjixinMlIP4eiO
16123 7zLYKWcXnYeHHWidalz7rj
16124 12TOa8pzhbLTWh1vxIsPMp
16125 2rYyq0b3XWwmWl0k727OVG
16126 74lnM5V6ecvoTPV0fvptx9
16127 6Epjc6EPgXsCKCyi6eAaNK
16128 2yI8omCrTAyrTcK7NRppu7
16129 3nc420PXjTdBV5TN0gCFkS
16131 2aRom6JjM2fH9dt971uewi
16132 5ToJzoVfLYyF7ohMOBNObi
16133 44VMIYttRMiruEmIObpz7P
16134 74ZovzRxHBvCq5yIwe8boh
16135 65yzeXye6OFzjelGCZz4lK
16136 4MZQ3lHA1TYO6yyedtmBYg
16138 3DnZfpfe

### Search by title (cleaned) and artist

In [40]:
for i in sr.loc[sr['track_id'].isnull(),:].index.values:
    # clean title
    title = ''.join(c for c in sr['title'][i] if c.isalnum() or c.isspace())
    #title = ' '.join(c for c in sr['title'][i] if c.isalnum()) ## special format of titles
    
    artist = sr['artist'][i]
    
    query = 'track:"%s" AND artist:%s AND year:%s&type=track&limit=1'%(title,
                                                                       artist,
                                                                       sr['year'][i])

    r = requests.get(BASE_URL + 'search?q=' + query, headers=headers).json()

    try:
        results = r['tracks']['items'][0]

        artists_i = getArtistNameList(results['artists'])

        for a in artists_i:
            # deal with middle name -> match only first and last name
            if (a.lower() in artist.lower()) or (artist.lower() in a.lower()) or\
            ((artist.split()[0].lower() in a.split()[0].lower()) and (artist.split()[-1].lower() in a.split()[-1].lower())):
                print(i, results['id'])
                sr['track_id'][i] = results['id']
                sr['release_date'][i] = results['album']['release_date']
                sr['spotify.name'][i] = results['name']
                sr['spotify.artists'][i] = artists_i
                break
    except:
        pass

18609 7upm07Y3Fx4AJs5G5OqG1A
18611 5rnqbmSmvqKzDsGZoqeuTN
18612 03WIqWZZSHnzJF3D18x8Gh
18616 4SFknyjLcyTLJFPKD2m96o
18623 4CPszpSnlbgnQLIQYyzIR2
18626 6Qp1DSekiuxHbFoSsg9JIk
18628 6WoKLs9kOA7I9RJtVYr6MQ
18634 5blHL2xSCGv00fr1DGuvEf
18640 7okHSkLBzwcGAhhQJeGgsF


### Traverse artists for multi-artist songs with title

In [41]:
for i in sr.loc[sr['track_id'].isnull(),:].index.values:
    title = sr['title'][i]
    
    # traverse artists for multi-artist songs
    for artist in re.split(', | & | X | x | and ', sr['artist'][i]):
        
        query = 'track:"%s" AND artist:%s AND year:%s&type=track&limit=1'%(title,
                                                                           artist,
                                                                           sr['year'][i])

        r = requests.get(BASE_URL + 'search?q=' + query, headers=headers).json()

        try:
            results = results = r['tracks']['items'][0]
            
            artists_i = getArtistNameList(results['artists'])

            for a in artists_i:
            # deal with middle name -> match only first and last name
                if (a.lower() in artist.lower()) or (artist.lower() in a.lower()) or\
                ((artist.split()[0].lower() in a.split()[0].lower()) and (artist.split()[-1].lower() in a.split()[-1].lower())):
                    print(i, results['id'])
                    sr['track_id'][i] = results['id']
                    sr['release_date'][i] = results['album']['release_date']
                    sr['spotify.name'][i] = results['name']
                    sr['spotify.artists'][i] = artists_i
                    break
            break
        except:
            pass
    

449 5j0SvoSBSzBaXGD0IQdmA4
1071 2dERV62dG0g16ACbb7P2S3
1247 3BzK0pce5dIZHpMXR3VPJ7
3248 2yi7HZrBOC4bMUSTcs4VK6
3319 68PD93zd4LzpxMWDxs5ugh
3755 7n4JJDiGCGoc01cn439ivX
4812 4ws6ARKJGwPHWOXucBpiDl
5012 49zAFEviq7WrEGWuYL2CtN
5387 3sib8YIkqyxKokUZ7zy0kj
6371 483XiZ5o13Cc1zoWV7jGml
6562 5xO5ON84ieWrb8wYz7j1v1
6615 3YB9cvd668HXBEq8rbBW8P
6663 2ELVVIbpucfOqGFC21Q4yR
6688 3GKDM1IEh0GWFMR7g5Lexo
6788 5wnfZ7VjDpZNyfJkqc9I1d
7606 7J0UbeV4G3iJISrFm1aWN7
7631 4LdKhiumggIeOPIVKOJ6eQ
8055 0FTzAmvMu2zKdp3ZrvlliF
8440 1DmnEYXa4WfbdhAPwNzgD8
8668 7jslhIiELQkgW9IHeYNOWE
8715 32k4TVAGwX2qQzdrkhRgYi
8961 2AGottAzfC8bHzF7kEJ3Wa
8968 0GR7iJLhj80KD5LkA14ZRn
9033 1IecEbWHvnC6kiwDutTUX1
9247 2QJnTfMpNG05KFf2E3gVIJ
9586 2ACys0pX0SjmpQmQWzp7wt
9601 27GmP9AWRs744SzKcpJsTZ
9724 2uTG00P1DkhB97pdITkl2O
9820 0tG61juk79S6dEQPVgwGgN
9858 0UIk2B5sfUKMBAkAyLZBiB
10116 76kyKtPLsFbQkdQ86QrkF4
10241 2bNxCVI4Fub8BAwr0CQDyd
10861 786tePtnXq0yIi3gm0xWhD
10925 4sm6HIt7yfcHKFHGgodW9o
11593 4ujX9kC3UUp26MG0tPhUCz
12272 4gmmRb6bZJ

### Search by title with artist traversal with title (cleaned)

In [43]:
for i in sr.loc[sr['track_id'].isnull(),:].index.values:
    # clean title
    title = ''.join(c for c in sr['title'][i] if c.isalnum() or c.isspace())
    #title = ' '.join(c for c in sr['title'][i] if c.isalnum()) ## special format of titles
    
    # traverse artists for multi-artist songs
    for artist in re.split(', | & | X | x | and ', sr['artist'][i]):
        # direct search by title and artist
        query = 'track:"%s" AND artist:%s AND year:%s&type=track&limit=1'%(sr['title'][i],
                                                                            artist,
                                                                            sr['year'][i])
        # not specify year
        ##query = 'track:"%s" AND artist:%s&type=track&limit=50'%(title, artist)

        r = requests.get(BASE_URL + 'search?q=' + query, headers=headers).json()
        
        try:
            results = r['tracks']['items'][0]

            artists_i = getArtistNameList(results['artists'])

            for a in artists_i:
                # deal with middle name -> match only first and last name
                if (a.lower() in artist.lower()) or (artist.lower() in a.lower()) or\
                ((artist.split()[0].lower() in a.split()[0].lower()) and (artist.split()[-1].lower() in a.split()[-1].lower())):
                    print(i, results['id'])
                    sr['track_id'][i] = results['id']
                    sr['release_date'][i] = results['album']['release_date']
                    sr['spotify.name'][i] = results['name']
                    sr['spotify.artists'][i] = artists_i
                    break
            break
        except:
            pass

### Search without specifying year

In [46]:
for i in sr.loc[sr['track_id'].isnull(),:].index.values:
    # clean title
    title = ''.join(c for c in sr['title'][i] if c.isalnum() or c.isspace())
    #title = ' '.join(c for c in sr['title'][i] if c.isalnum()) ## special format of titles
    
    # traverse artists for multi-artist songs
    for artist in re.split(', | & | X | x | and ', sr['artist'][i]):
        # direct search by title and artist
        #query = 'track:"%s" AND artist:%s AND year:%s&type=track&limit=50'%(title,
                                                                            #artist,
                                                                            #sr['year'][i])
        # not specify year
        query = 'track:"%s" AND artist:%s&type=track&limit=1'%(title, artist)

        r = requests.get(BASE_URL + 'search?q=' + query, headers=headers).json()
        
        try:
            results = r['tracks']['items'][0]
            
            artists_i = getArtistNameList(results['artists'])

            for a in artists_i:
                # deal with middle name -> match only first and last name
                if (a.lower() in artist.lower()) or (artist.lower() in a.lower()) or\
                ((artist.split()[0].lower() in a.split()[0].lower()) and (artist.split()[-1].lower() in a.split()[-1].lower())):
                    print(i, results['id'])
                    sr['track_id'][i] = results['id']
                    sr['release_date'][i] = results['album']['release_date']
                    sr['spotify.name'][i] = results['name']
                    sr['spotify.artists'][i] = artists_i
                    break
            break
        except:
            pass

12 5st6ogCAEzsPbqOJfoJfbT
41 2zW1XlHAazTViSVM60yh6z
51 3IJGQEgU7pZOj1uDHIPfwY
55 6PDDLs0XDsWyhJxIqsgzWW
61 4sT8kcliEkMJBUTQqKOIUb
82 2FOLVyfKFWzS0F5WzC7NLD
86 5QrqXOTpWw4MfoDlHfCdBL
88 0USZx50eApN045zIIyjePN
112 6C1BUMo3Eq3Xudv2Oo2CLr
134 3Rh3zDYanZFHaeXFT4Asps
140 2KndZ38hCNVk3Jmb0l6uDC
168 09XGGn9lYHT8Mcsu1oiOJP
175 7LrgiRjCm2oKcuJGbmk0pg
184 2HHiH52OPxe8qyVO7N4uH7
204 735SArN9pqYYzYg0leQ8nm
218 64y85LeHY8Z6OlTOM9cpKD
219 3bwCMbwDZVtvJDnUTQIdCX
220 2EUHzmBZux7SSra71cGkIB
258 3DwN8YfAtvJh4c5u8DZPen
266 0NoKPhyT2DZ8yxtVvT0Bad
311 4IFTlLCrZsToaJ3cYlRdpb
314 71fbIsRGkm6WG9CTGyvEum
358 78GjajEEFelJf4Ibk3q7GL
361 6mEDMe0zG5pYrsRxizjauk
362 0zmMSFU9c46I2mmw34a4XF
366 1rIvIifhYqdH5OHQjvbykW
367 22u0bVblodXzifcToTjrC1
386 2Fvvez4jjuTF5vrDESR2qV
408 331q5cdwo9aq7ih6bSpoOa
421 023OVLNzXhX0j7CxswUt6D
428 3PNYZeKsUfm1ks7giH1Myx
433 6plT7nFGiXKSBP9HFSI4ef
438 5cj2g0s91RSoKpkAyqs2gQ
469 2kWowW0k4oFymhkr7LmvzO
482 6I5cQVYvoc5dqpyYvYHpSw
487 1SeZHSmP09eYHXfMoGNrLM
502 3mIinBcQwppTzuPPcNNRhz
514 6mH3q

3924 2xUHiyGeGrWmmfv7NbDTWC
3931 27ZhKzjHoE0UUlyMXu0ZFa
3935 2jNSYcYDifZJAAi5gMVJR5
3948 2Ih0U9yUuY0oU2In3h21If
3963 3JMyRsMQIeRXSWwFbO5gLn
3982 0ciA43AxH3rZASCUBmYvPJ
3988 7lG4154Md1Kw7BMg56Pt4s
4029 5OFnCw227Q1wr8cEEftzv8
4035 20JJlcvJo0IXffpU5dRqDB
4066 20mm07wltfpLJ6vKbx01da
4072 6yjsa6c8nSGQqXF12vFSpp
4084 2kleQWygkA5yDpt4zIfWzd
4102 6IDDwI0YOCAUDhMZltQekS
4113 5IXV375WDokSNjZfR4GgnU
4114 2RAXAF7qjvn0FsRuMMTlBx
4122 2MDueMvPkUTpU6Xkmeinln
4123 1aQuEPiDbYD0n3ob8YKBQn
4158 5SsR3wtCOafDmZgvIdRhSm
4194 3h7pI8oM2CpXiNSZbQuEtV
4197 63FrXif0Pdu4NAPvTh87mw
4221 2Xaw1RrXP3mdooKWysuRJ8
4234 6np7WP2rTg4ofHFyRWCM0G
4239 5XEZx88UTiwb01Hg4GRi6H
4264 6UmZNpLWzkRWeln4KG0K84
4304 4DActPOAtak2m8meZeMt3B
4355 5Amm6v8Af2mkVzoso3XZcj
4382 3TY6q0ZeSVg9byfug6KKrU
4405 6wQswh6JNND7WwelaYyNhJ
4409 5K8cyMKcnG4rx5ocW3u7OW
4416 274EG72CmKSa3mWBsacoL7
4419 2nIulsLMiP2SMTDrtxfQXX
4438 0Yp3wCUpjrG7NWWPDpmQXm
4442 2FRNaqLU6UFv9kANvg8D7c
4443 0nfUOSvcUfntw9a44vtlOb
4463 0fAJ7GyI7NHDD3YhuNgdDP
4495 4XOVGSQjmxtSbt3

7714 4rUTAkOU7OkVGeQuyQCHhF
7719 7m88eIVGz5yWoikkHnoVIf
7733 3OqwM9CRqLskni9acTwuj1
7739 7fLHguF12laXV09NVTrlSD
7741 3TmyCY31Q5MEDV1nWWrYAR
7755 4eQznxonJdR6NEksFNHMCT
7758 0rZF3zZKA2oVDiwW46WL6l
7761 5C4NMi4fPpdFPnMZVsyG8H
7765 0ITvLv9ys64WXJjJ8pLkJi
7794 7J0mZi4vIRqBR9pjAt90nX
7807 3uF2FEJ9Y8KYnSvb0B1kWS
7820 0cUYHAciIIom2n04j9p5qj
7830 1IDF4jYiuI17U4BW23EoXV
7840 4uuddHEP8cr9EMAVkSNI9N
7855 7rnhxvGrHEEu1KFzb7fpxh
7889 1fBvk6KaOsuUfXaKU1M4M9
7896 5teGX9dkVCZOuao7dOWs7l
7897 39J9gLwAyM6n2c2vFVVvdV
7926 2U9kDk5mlHYunC7PvbZ8KX
7929 6ODoMQe0JwO6bgGV1JqAUC
7957 1OWv06xSpcaNE1eQwUJPvF
7965 1mAPo7CPlVL3HDT1ZRqhLr
7966 0hjANar01OL1RWox3nYT7k
7981 7F5J5cfDnF0h9s69zoeV4O
7982 5oNlBs0y1lB8tBA65E7abl
8004 5XWqdj5nPNVeM2TxIiyY5W
8025 4eGVm6LoWxE0oyVGAmgNAw
8043 3Mi8mCnvcBPdiPgmHYWEow
8048 0KUQVPAXbCrvVrWslGZXSh
8058 0588kHJ34DF7ZvTyuo4Dnk
8083 5uqphUYsR45kPYplrYSAQh
8100 0v60rM2HLSatgUyVZDrLwm
8101 0k5hoseEJnCAbpRh38dNoI
8105 053s9IvqFRoOi3Ud1epOD4
8107 68m2FTTwwBc3a88b9w3UOl
8115 0nrwXo4inMMLcV3

11755 2yQZwi1P8AkkxxFhQ8rMEK
11779 6opSOlCOFqO3yDje0qI9qn
11784 13LI79uAnn9qb1IdHQb7mr
11803 3YVg4yurS4YaExuUDtSoYp
11833 12EKNVYavyUOF2AGeiisNl
11843 5C6uh95eAL0RBTSGXKQwvk
11847 35lBL83KZl80dHivikZKEq
11849 0wb0X7n3cAKeajIOguyd9t
11859 0ZUCKcWdFABozlSmojDyLZ
11872 3V7WIBmAFOpu2QIrkYoUeP
11878 09iyGiljLWRYXdRazFkdtJ
11889 2jB9QaPJyVdz2Q0Va8rrnU
11897 1wIDxpXCRnHmQJOvSsVIP1
11915 32jNxnHEdW0R152i2MFLIC
11920 1MAtajbUK7QNlvMIlDAxMz
11923 4VA9tLqZLmQ3HBxO5I992Z
11925 5NbBEzfLZc6anPJYlxNpG1
11935 4Az17HfqonKSWNsaLgTBeK
11968 1Cz73d6PqsQwSxXhfLrPOk
11998 3sdiDU9CIyic6Ts2HO2ucD
11999 3wQjHSfCaPyPatvz9sn4w4
12006 2f4FDuRjuV1yXfpoG49ABM
12021 5HqpTLBs9TbMPlUb5MMO60
12025 15Lw8Uofm03raFJeuVrQ0k
12029 5dViLbqgaPvvLN6IS3F7HM
12032 3rsOJ5Nbmr7pWxY1p6ZDxk
12049 6r17TK9vMMuwTTL2p7O3Ac
12064 02hx6RMAfqOVdG3C0uyTFg
12072 6eUClMzq8c9cKBXd6NrgVN
12089 0ceXGMDXYHcRIYRPnuuY0u
12096 152lZdxL1OR0ZMW6KquMif
12100 5hIw8IeZ1UcxB8yZrEy9rc
12102 2Wj1AZdBPMA1mkuuoTfOPg
12133 3YM0vEH7taFj7OLW3dWfiu
12156 6XBkL7qS

17312 4Im6GRj17qa7NW76OsJh1s
17372 558ubozatFQlweufvHa3Ip
17385 0uLPKDuIhkIhttiwezNdiG
17455 1iyE0UwRtLUBf9OqLfFGSp
17457 7eOkHIcaoUhngxxypM5Ajr
17494 6NrWdxWR0sj4gPAOU2w2tu
17558 6jGyDvL1Rf4XfwTb25o1HN
17577 4drTFbY9KJIvllrqVcJvLi
17610 5T8OICEqdqbx8Uz7L5abkH
17630 2ce6x2fCzlgI2fnCAGJDUu
17644 47qZzTLLJcuD69AIKnquo4
17652 2cytBOLpwFRX7J9URCrFIe
17694 1KcOU0q9AMXV9xi4tSzzCc
17695 3hImfCYVAHQBtONi06hqs8
17746 2VPmBOuy7ZAOFSzKwW2IEt
17770 1DVWkJEjgVStgU2EbR9vpz
17846 0o1xsthxqPUiKqVqRN0r56
17876 5pSRBmJhkfZm1eilG3VW3L
17899 29aVsOfrhR9chb2TzKW2dh
17917 0LybHog3IZ7ywvkgkXrBW7
17939 6BK8BcFxMFr3JVUaAOZsYC
17964 15C4TnrrVdym7UykxQIOTZ
17982 5mcxeKzIS8aDbz335w7Bq0
18061 6QZP7wxsor0vZsI9dkKZk2
18082 5avVDmK9FVJpd3MDO8I3Zl
18223 2kFHjWko1il6O9L3eK9IzG
18224 1fckqKAI9ug7U1DgQrrOop
18338 6MGRweoLPBAaPcXTMpMWsA
18419 4jkwu5KQbXXrbAZzVmt5fE
18466 5XwxYaLbKnT54197S5Hz3z
18508 645Exr2lJIO45Guht3qyIa
18548 205fvU0chRUNVGLu71zD6B
18565 6LVzOLJkzG3C4hLNOdaBry
18610 3Gyl1BvC41QD8rXEMfSQWY
18642 6xzywdfa

### Search title with special format

In [102]:
for i in sr.loc[sr['track_id'].isnull(),:].index.values:
    # clean title
    #title = ''.join(c for c in sr['title'][i] if c.isalnum() or c.isspace())
    title = ' '.join(c for c in sr['title'][i] if c.isalnum()) ## special format of titles
    
    # traverse artists for multi-artist songs
    for artist in re.split(', | & | X | x | and ', sr['artist'][i]):
        # direct search by title and artist
        query = 'track:"%s" AND artist:%s AND year:%s&type=track&limit=1'%(title,
                                                                            artist,
                                                                            sr['year'][i])
        # not specify year
        ##query = 'track:"%s" AND artist:%s&type=track&limit=50'%(title, artist)

        r = requests.get(BASE_URL + 'search?q=' + query, headers=headers).json()
        
        try:
            results = pd.json_normalize(r['tracks']['items']).sort_values('album.release_date', ignore_index=True)

            for j in range(len(results)):
                artists_i = getArtistNameList(results['artists'][j])

                for a in artists_i:
                    # deal with middle name -> match only first and last name
                    if (a.lower() in artist.lower()) or (artist.lower() in a.lower()) or\
                    ((artist.split()[0].lower() in a.split()[0].lower()) and (artist.split()[-1].lower() in a.split()[-1].lower())):
                        print(i, results['id'][j])
                        sr['track_id'][i] = results['id'][j]
                        sr['release_date'][i] = results['album.release_date'][j]
                        sr['spotify.name'][i] = results['name'][j]
                        sr['spotify.artists'][i] = artists_i
                        break
                break
            break
        except:
            pass

3129 4L0HMiE70JyFSfOsnoHGbj
4534 3Eq7yD58dIXqOgw1j7NFhY
5579 6dm7uLUcCq7bVDjgMWhHLf
8228 2i2CmkYlqh27qsac9mPZLA
9091 7kWks2sTkgY8ThFSwRTrHg
10969 2L7LTut7s6BYo9tcjidSSu
19169 0XS2rOeAPNEoi23fS7K5TO
19392 4n6NDfYake476trCjJRNl0
19404 5R691ipUYRDYW6ehapjoj6
19422 2cnKST6T9qUo2i907lm8zX
19430 1d7q712nXjG98HiwHk7HFS
19694 0GAyuCo975IHGxxiLKDufB
19753 1D3z6HTiQsNmZxjl7F7eoG
19819 5W8jRrZ6tWrTrqnKRtIQBf
19822 57ZUX6TNyKLBydAdVVd02x
19960 1QL7nSDZCwZMnbisV4KOXt


### Check title differences

In [108]:
'''
title_dif_idx = []
for i in sr.loc[~sr['track_id'].isnull()].index.values:
    if sr['title'][i].lower() not in sr['spotify.name'][i].lower():
        title_dif_idx.append(i)

title_dif = sr.iloc[title_dif_idx]
#title_dif.to_excel('new_spotify_correct_title_dif.xlsx')

new_title_dif = pd.read_excel('title_dif.xlsx')
sr.iloc[new_title_dif['Unnamed: 0'].values, [3,4,5,6]] = new_title_dif.iloc[:,[4,5,6,7]].values
'''

"new_title_dif = pd.read_excel('title_dif.xlsx')\nsr.iloc[new_title_dif['Unnamed: 0'].values, [3,4,5,6]] = new_title_dif.iloc[:,[4,5,6,7]].values"

### Check duplicate track IDs

In [75]:
'''
dup = sr[sr.duplicated(['track_id'],keep=False)]
dup = dup.loc[~dup['track_id'].isnull()]
dup
#dup.to_excel('new_spotify_correct_dup.xlsx')

new_dup = pd.read_excel('new_spotify_correct_dup2.xlsx')
sr.iloc[new_dup['Unnamed: 0'].values, [3,4,5,6]] = new_dup.iloc[:,[4,5,6,7]].values
'''

### Check year and release_date discrepancies

In [78]:
'''
year_dif = sr.loc[(~sr['release_date'].isnull())]
year_dif = year_dif.loc[year_dif['release_date'].str.len()>4]
year_dif = year_dif.loc[abs(year_dif['year']-year_dif['release_date'].str[:4].astype(int))>=1]

#year_dif.to_excel('new_spotify_year_dif.xlsx')

new_year_dif = pd.read_excel('year_dif.xlsx')
sr.iloc[new_year_dif['Unnamed: 0'].values, [3,4,5,6]] = new_year_dif.iloc[:,[4,5,6,7]].values
'''

### Null values

In [ ]:
'''sr.loc[sr['track_id'].isnull()].to_excel('new_spotify_correct_null.xlsx')
new_null = pd.read_excel('new_spotify_correct_null.xlsx')

new_null['track_id'] = new_null['track_id'].str.replace('https://open.spotify.com/track/','')

for i in range(len(new_null)):
    if new_null['track_id'][i] is not np.nan:
        new_null['track_id'][i] = new_null['track_id'][i].split('?si=')[0]
        
sr.iloc[new_null['Unnamed: 0'].values, [3,4,5,6]] = new_null.iloc[:,[4,5,6,7]].values'''

In [104]:
#sr.dropna(subset=['track_id']).to_csv('get_id/song_release_ID.csv', index=False)
#sr.dropna(subset=['track_id']).to_excel('get_id/song_release_ID.xlsx', index=False)

## Get track information

In [93]:
sr = pd.read_csv('song_release_ID_filled.csv').iloc[:,0:4]

# initialize track_info dataframe
track_info = pd.DataFrame(columns=['artists', 'available_markets', 'disc_number', 'duration_ms',
       'explicit', 'href', 'id', 'is_local', 'name', 'popularity',
       'preview_url', 'track_number', 'type', 'uri', 'album.album_type',
       'album.artists', 'album.available_markets',
       'album.external_urls.spotify', 'album.href', 'album.id', 'album.images',
       'album.name', 'album.release_date', 'album.release_date_precision',
       'album.total_tracks', 'album.type', 'album.uri', 'external_ids.isrc',
       'external_urls.spotify'])


for i in range(len(track_info), len(sr), 50):
    if not i+50 > len(sr):
        ids = ','.join(sr['track_id'][i:i+50])
    else:
        ids = ','.join(sr['track_id'][i:])
        
    r = requests.get(BASE_URL + 'tracks?ids=' + ids , headers=headers)
    
    track_info = pd.concat([track_info, pd.json_normalize(r.json()['tracks'])],
                               ignore_index=True)

In [94]:
def getArtistName(artists_list):
    artists = []
    for a in artists_list:
        artists.append(a['name'])
    return ', '.join(artists)

track_info_clean = track_info[['name','artists','id','album.release_date']]

for i in range(len(track_info_clean)):
    track_info_clean['artists'][i] = getArtistName(track_info_clean['artists'][i])

In [100]:
sr.merge(track_info_clean, left_on='track_id', right_on='id').to_csv('song_release_ID_spotify.csv', index=False)